<a href="https://colab.research.google.com/github/yonatanlou/study_projects/blob/main/Statistics_lab/labs/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 3

In [ ]:
# Import modules 
import numpy as np
import pandas as pd 
import statsmodels.api as sm 
import seaborn as sns
import matplotlib.pyplot as plt

import random
random.seed(42)


pd.get_option("display.max_columns")

0

In [ ]:
data_url = "https://raw.githubusercontent.com/yonatanlou/study_projects/main/Statistical_learning_and_data%20analysis/votes_per_ballot_2021.csv"
votes_per_ballot = pd.read_csv(data_url,  encoding = 'iso-8859-8', index_col='שם ישוב')

df = votes_per_ballot


#### Q1

#### Data preparation

In [ ]:
parties_dict ={'אמת': 'עבודה',
 'ב': 'ימינה',
 'ג': 'יהדות התורה',
 'ודעם': 'הרשימה המשותפת',
 'ט': 'הציונות הדתית',
 'כן': 'כחול לבן',
 'ל': 'ישראל ביתנו',
 'מחל': 'הליכוד',
 'מרצ': 'מרצ',
 'עם': 'רעם',
 'פה': 'יש עתיד',
 'שס': 'שס',
 'ת': 'תקוה חדשה'}
##relevant 13 parties

In [ ]:
total_counted = df['כשרים'].sum()

isInKnesset = (df.sum(axis=0)/total_counted > 0.0325).to_dict()
inKnessetParties = [i for i in isInKnesset.keys() if isInKnesset[i] is True]
knesset_parties = df[inKnessetParties]

In [ ]:
# Here I adjusted the dataframe to be the every cell is the supporters of each party in each ballot

total_bzb = df['בזב'].sum()
total_kosher = df['כשרים'].sum()
knesset_parties = (knesset_parties*total_bzb)/total_kosher
knesset_parties = knesset_parties.round()
knesset_parties = knesset_parties.loc[~knesset_parties.index.str.fullmatch('מעטפות חיצוניות')]
knesset_parties = knesset_parties[knesset_parties.columns[8:]]
 ##Dataset fixed to supporters as required and withwout מעטפות חיצוניות.


The main function:

In [ ]:
# df2a = df
# df2a['percent_A'] = df2a['כשרים']/df2a['בזב']
# df2a = df2a.loc[~df2a.index.str.fullmatch('מעטפות חיצוניות')]
# per_A = df2a['percent_A']
# knesset_parties


In [ ]:
#works for A and B, maybe C
def binomialSample(df, v):
  df_numpy = df.to_numpy().astype(int) #df in numpy for func
      
  if v.size == 12127 : ##sim1
    v_df = pd.concat([v] * (len(knesset_parties.columns)), axis=1, ignore_index=True)# make a df of percenteges
    v_numpy = v_df.to_numpy().astype(float) #df of percent to numpy
    f1_result = np.random.binomial(df_numpy,v_numpy)
    return f1_result
  
  if v.size == 13 :#sim2
    v_df = pd.concat([v] * (len(knesset_parties)), axis=0, ignore_index=True)
    v_numpy = v_df.to_numpy().astype(float) #df of percent to numpy
    f1_result = np.random.binomial(df_numpy,v_numpy)
    return f1_result

  ##sim3
  f1_result = np.random.binomial(df_numpy,v)
  return f1_result
 

#Q2


#### Simulation 1

In [ ]:
##creating vector of % in each ballot to send to the sim
df2a = votes_per_ballot
df2a['percent_A'] = df2a['כשרים']/df2a['בזב']
df2a = df2a.loc[~df2a.index.str.fullmatch('מעטפות חיצוניות')]
per_A = df2a['percent_A']

In [ ]:
df_sim1 = knesset_parties*0 #df that will have the results after each sim
for x in range(50) :
  df_sim1 = df_sim1 +binomialSample(knesset_parties,per_A)
df_sim1 = df_sim1/50
df_sim1  

,אמת,ב,ג,ודעם,ט,כן,ל,מחל,מרצ,עם,פה,שס,ת
שם ישוב,,,,,,,,,,,,,
אבירים,46.96,0.00,0.00,2.82,1.80,15.92,2.24,4.82,30.26,0.0,27.16,0.56,8.30
אדירים,0.78,7.70,4.64,0.00,10.02,3.12,0.00,97.28,0.00,0.0,2.20,70.32,6.98
אושה,77.34,7.16,0.00,2.92,0.00,39.46,8.46,26.46,21.60,0.0,92.38,0.00,15.60
אחווה,16.30,4.94,0.00,0.00,4.74,43.52,3.08,31.56,12.00,0.0,43.94,2.36,9.42
איבים,0.00,1.54,0.00,0.00,4.60,2.02,0.00,7.34,0.00,0.0,0.00,0.38,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
קורנית,100.94,20.22,0.00,0.74,3.26,84.06,11.66,36.64,113.52,0.0,172.88,0.00,16.78
שורשים,60.78,19.24,0.00,2.72,0.70,38.90,6.80,33.04,21.94,0.0,83.58,0.00,25.64
שכניה,107.30,17.82,0.00,2.96,4.10,64.88,12.88,14.74,72.60,0.0,131.30,0.70,38.58


#### Simulation 2

preperation to send to func
Icreated the vector of voting % per party based on a few assumptions.
1) ultra religous vote alot because that what the rabbis tell them to.
2) Arabs voted in small percentges
3) the samller the party is, more likly for a supporter to actually, and the same on the other direction.
4) new parties will get a high percentge of votes from their supporters.

In [ ]:
knesset_parties
per_b = [0.65,0.65,0.75,0.4,0.7,0.6,0.7,0.5,0.75,0.55,0.55,0.7,0.65]

per_b2 = pd.DataFrame(per_b,columns=['Percentge of vote'])
per_b2 = per_b2.T
type(per_b2)

pandas.core.frame.DataFrame

In [ ]:
df_sim2 = knesset_parties*0 #df that will have the results after each sim
for x in range(50) :
  df_sim2 = df_sim2 +binomialSample(knesset_parties,per_b2)
df_sim2 = df_sim2/50
df_sim2  


,אמת,ב,ג,ודעם,ט,כן,ל,מחל,מרצ,עם,פה,שס,ת
שם ישוב,,,,,,,,,,,,,
אבירים,45.16,0.00,0.00,1.70,2.32,13.84,2.16,3.54,34.62,0.0,22.36,0.66,7.98
אדירים,0.60,6.74,4.62,0.00,9.16,2.22,0.00,62.80,0.00,0.0,1.80,63.28,5.76
אושה,70.10,6.38,0.00,1.50,0.00,33.36,8.58,18.30,22.40,0.0,69.74,0.00,14.36
אחווה,14.30,3.90,0.00,0.00,4.12,32.02,2.50,19.22,11.50,0.0,30.52,1.96,7.60
איבים,0.00,3.90,0.00,0.00,10.46,3.96,0.00,12.42,0.00,0.0,0.00,0.68,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
קורנית,86.96,17.50,0.00,0.44,2.72,64.92,10.96,22.90,109.48,0.0,123.12,0.00,14.26
שורשים,58.00,18.98,0.00,1.52,0.66,34.56,6.98,24.68,23.16,0.0,66.56,0.00,23.82
שכניה,97.68,16.56,0.00,1.62,4.16,55.10,12.38,10.66,74.40,0.0,99.96,0.68,35.50


#### Simulation 3

data preperation - building matrix

In [ ]:
import itertools
per_b = [0.65,0.65,0.75,0.4,0.7,0.6,0.7,0.5,0.75,0.55,0.55,0.7,0.65]
import random
def perm_generator(seq):
    seen = set()
    length = len(seq)
    while True:
        perm = tuple(random.sample(seq, length))
        if perm not in seen:
            seen.add(perm)
            yield perm


rand_perms = perm_generator(per_b)

j = [next(rand_perms) for _ in range(12127)]






In [ ]:
per_3 = np.asarray(j)
type(per_3)


numpy.ndarray

In [ ]:
df_sim3 = knesset_parties*0 #df that will have the results after each sim
for x in range(50) :
  df_sim3 = df_sim3 +binomialSample(knesset_parties,per_3)
df_sim3 = df_sim3/50
df_sim3  


,אמת,ב,ג,ודעם,ט,כן,ל,מחל,מרצ,עם,פה,שס,ת
שם ישוב,,,,,,,,,,,,,
אבירים,37.76,0.00,0.00,2.86,1.00,17.56,2.16,4.32,29.90,0.0,29.98,0.76,6.40
אדירים,0.62,6.36,2.38,0.00,9.62,2.64,0.00,86.34,0.00,0.0,1.34,54.48,6.70
אושה,70.36,6.64,0.00,2.66,0.00,38.86,7.66,19.74,16.60,0.0,50.82,0.00,10.42
אחווה,13.22,3.88,0.00,0.00,3.78,27.00,2.38,29.88,5.70,0.0,41.98,2.08,8.48
איבים,0.00,2.32,0.00,0.00,10.24,5.26,0.00,13.40,0.00,0.0,0.00,0.76,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
קורנית,81.38,20.28,0.00,0.60,2.90,81.60,8.32,34.26,58.52,0.0,111.48,0.00,14.50
שורשים,62.56,19.78,0.00,2.10,0.76,36.94,7.52,18.68,19.94,0.0,67.12,0.00,27.78
שכניה,112.70,16.62,0.00,2.24,4.24,59.04,8.70,8.38,71.80,0.0,125.66,0.80,30.46


#### 2.b

Expected Value for each party in each simulation

In [ ]:
#df_sim1
sums = df_sim1.select_dtypes(pd.np.number).sum().rename('Simulation 1')
df_sim1 = df_sim1.append(sums)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [ ]:
#df_sim2
sums2 = df_sim2.select_dtypes(pd.np.number).sum().rename('Simulation 2')
df_sim2 = df_sim2.append(sums2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [ ]:
#df_sim3
sums3 = df_sim3.select_dtypes(pd.np.number).sum().rename('Simulation 3')
df_sim3 = df_sim3.append(sums3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


number of total votes for the top 13 parties, in each of the 3 simulations

In [ ]:
merged = df_sim1.append(sums2)
merged = merged.append(sums3)
merged = merged.iloc[12127:12130]


#### 2b final answer

estimators for each party in each simulation (the mean from each 50 samples).

In [ ]:
res = merged.div(merged.sum(axis=1), axis=0)
res

,אמת,ב,ג,ודעם,ט,כן,ל,מחל,מרצ,עם,פה,שס,ת
שם ישוב,,,,,,,,,,,,,
Simulation 1,0.062274,0.063081,0.065753,0.037811,0.053760,0.071217,0.051730,0.243449,0.047061,0.030554,0.148337,0.076326,0.048648
Simulation 2,0.064709,0.064664,0.074630,0.034187,0.058942,0.067774,0.065676,0.205352,0.057760,0.037549,0.130274,0.086885,0.051597
Simulation 3,0.059456,0.059879,0.059574,0.051472,0.050219,0.067692,0.056113,0.244955,0.046118,0.040713,0.142060,0.074246,0.047502


#### 2.c

####Average

multiply each cell by the suppoters of total supporters of taht party


In [ ]:
knesset_parties_avg = knesset_parties.sum(axis=0)
knesset_parties_avg = knesset_parties_avg.T
knesset_parties_avg.sum()

5849530.0

In [ ]:
qhat_avg = knesset_parties_avg*res
qhat_avg.iloc[0]

אמת      21702.326124
ב        21968.247769
ג        22901.649551
ודעם     11315.103956
ט        15842.082216
כן       28145.690948
ל        16989.349187
מחל     349910.961285
מרצ      12686.441365
עם        7298.148849
פה      122946.549015
שס       33159.333877
ת        13518.531044
Name: Simulation 1, dtype: float64

###variance


Because it is a binomi, we will multiply the expected value by 1-p of each value

In [ ]:
minus_p_df = 1-res 
q_hat_var = qhat_avg*minus_p_df
q_hat_var

,אמת,ב,ג,ודעם,ט,כן,ל,מחל,מרצ,עם,פה,שס,ת
שם ישוב,,,,,,,,,,,,,
Simulation 1,20350.833976,20582.478323,21395.788007,10887.272435,14990.410174,26141.232752,16110.483384,264725.544640,12089.403837,7075.164320,104709.034444,30628.429177,12860.887175
Simulation 2,21091.669085,21063.608036,24053.543916,9881.062325,16345.302320,24969.541017,20152.693192,234543.802019,14671.297829,8632.242270,93908.809880,34467.176936,13598.454929
Simulation 3,19488.361014,19604.749001,19513.163022,14610.531674,14055.510089,24941.415870,17394.620582,265833.036532,11858.952698,9328.867546,101017.458233,29861.078786,12573.182122


###Bias

To calculate Bias, from the formula, we will reduce from q hat, the actual q.

In [ ]:
total_counted = df['כשרים'].sum()

isInKnesset = (df.sum(axis=0)/total_counted > 0.0325).to_dict()
inKnessetParties = [i for i in isInKnesset.keys() if isInKnesset[i] is True]

In [ ]:
actual_Q = df[inKnessetParties]
actual_Q = actual_Q.loc[~actual_Q.index.str.fullmatch('מעטפות חיצוניות')]
actual_Q = actual_Q[actual_Q.columns[8:]]
actual_Q ##Real results for 13 parties



,אמת,ב,ג,ודעם,ט,כן,ל,מחל,מרצ,עם,פה,שס,ת,percent_A
שם ישוב,,,,,,,,,,,,,,
אבירים,47,0,0,3,2,16,2,5,31,0,27,1,8,0.678733
אדירים,1,7,4,0,9,3,0,84,0,0,2,61,6,0.769231
אושה,72,7,0,3,0,37,8,24,20,0,86,0,15,0.722513
אחווה,14,4,0,0,4,36,3,27,10,0,37,2,8,0.792350
איבים,0,4,0,0,10,5,0,17,0,0,0,1,0,0.289855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
קורנית,89,18,0,1,3,73,10,32,98,0,150,0,15,0.767188
שורשים,60,19,0,3,1,38,7,32,21,0,81,0,25,0.685579
שכניה,101,17,0,3,4,61,12,14,68,0,121,1,36,0.721232


In [ ]:
temp_df = actual_Q.select_dtypes(pd.np.number).sum().rename('testbias')

temp_df #Real Q



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


אמת          234498.00000
ב            234143.00000
ג            234624.00000
ודעם         201397.00000
ט            198511.00000
כן           265521.00000
ל            220887.00000
מחל          963856.00000
מרצ          181815.00000
עם           160437.00000
פה           556110.00000
שס           292044.00000
ת            187084.00000
percent_A      7375.43511
Name: testbias, dtype: float64

###all of the bias are negative. seems like a mistake, please have a look.

In [ ]:
q_bias = qhat_avg - temp_df
q_bias

,percent_A,אמת,ב,ג,ודעם,ט,כן,ל,מחל,מרצ,עם,פה,שס,ת
שם ישוב,,,,,,,,,,,,,,
Simulation 1,NaN,-212795.673876,-212174.752231,-211722.350449,-190081.896044,-182668.917784,-237375.309052,-203897.650813,-613945.038715,-169128.558635,-153138.851151,-433163.450985,-258884.666123,-173565.468956
Simulation 2,NaN,-211947.081890,-211623.159280,-208630.557041,-191166.171946,-181141.931119,-238736.144903,-199317.730573,-668701.573464,-166244.336666,-151467.983421,-448134.872619,-254297.195018,-172745.726003
Simulation 3,NaN,-213777.687653,-213289.558113,-213874.726449,-185993.624828,-183712.309146,-238768.677030,-202458.287697,-611780.261281,-169382.689548,-150712.210367,-438365.764482,-259788.038942,-173883.779470


###MSE

###Doesn't seem right as well, based on bias probably because of that.

To calculate MSE, as formula, we will add to the var, the squre of the bias

In [ ]:
q_mse = q_hat_var + q_bias**2
q_mse

,percent_A,אמת,ב,ג,ודעם,ט,כן,ל,מחל,מרצ,עם,פה,שס,ת
שם ישוב,,,,,,,,,,,,,,
Simulation 1,NaN,4.528202e+10,4.501815e+10,4.482638e+10,3.613114e+10,3.336795e+10,5.634706e+10,4.157427e+10,3.769288e+11,2.860448e+10,2.345151e+10,1.876307e+11,6.702130e+10,3.012498e+10
Simulation 2,NaN,4.492159e+10,4.478438e+10,4.352673e+10,3.654452e+10,3.281242e+10,5.699497e+10,3.972758e+10,4.471620e+11,2.763719e+10,2.294256e+10,2.008250e+11,6.466710e+10,2.984110e+10
Simulation 3,NaN,4.570092e+10,4.549246e+10,4.574242e+10,3.459364e+10,3.375023e+10,5.701051e+10,4.098938e+10,3.742754e+11,2.869051e+10,2.271418e+10,1.921646e+11,6.748986e+10,3.023558e+10


###2D


function for 2d, based on solution of lab 2.

In [ ]:
# def party_bar(q_mean, p, q, sd_q, sd_p):
#         n = 13 # number of parties
#         width = 0.3
        
#         rev_names = [name[::-1] for name in list(names)]
        
#         # bar plot:
#         fig, ax = plt.subplots()  # plt.subplots()
#         q_mean_bar = ax.bar(np.arange(n), q_mean, width, yerr = sd_q, capsize=3, color = "b")
#         q_bar = ax.bar(np.arange(n) + 2*width, q, width, color = "g")
#         p_bar = ax.bar(np.arange(n) + width, p, width, yerr = sd_p, capsize=3, color = "r")
       
        
#         # legend
#         ax.set_ylabel('Votes percent')
#         ax.set_xlabel('Parties Name')
#         ax.set_title('Voters percentage by party - real data VS simulation VS correction')
#         ax.set_xticks(np.arange(n))
#         ax.set_xticklabels(names, rotation = -90)
#         ax.legend((q_mean_bar[0], q_bar[0],p_bar[0]), ("q hat", "q", "p"))
#         plt.xticks(rotation=45)
#         ax.set_xticklabels(rev_names, fontsize=10)
#         plt.show()
        
#         return fig, ax


In [ ]:
def party_bar(q_mean, p, q, sd_q, sd_p):
        n = 13 # number of parties
        width = 0.3
        
        rev_names = [name[::-1] for name in parties]
        
        # bar plot:
        fig, ax = plt.subplots()  # plt.subplots()
        q_mean_bar = ax.bar(np.arange(n), q_mean, width, yerr = sd_q, capsize=3, color = "b")
        q_bar = ax.bar(np.arange(n) + 2*width, q, width, color = "g")
        p_bar = ax.bar(np.arange(n) + width, p, width, yerr = sd_p, capsize=3, color = "r")
       
        
        # legend
        ax.set_ylabel('Votes percent')
        ax.set_xlabel('Parties Name')
        ax.set_title('Voters percentage by party - real data VS simulation VS correction')
        ax.set_xticks(np.arange(n))
        ax.set_xticklabels(parties, rotation = -90)
        ax.legend((q_mean_bar[0], q_bar[0],p_bar[0]), ("q hat", "q", "p"))
        plt.xticks(rotation=45)
        ax.set_xticklabels(rev_names, fontsize=10)
        plt.show()
        
        return fig, ax


In [ ]:
parties = ['עבודה', 
           'ימינה',
           'יהדות התורה',
           'הרשימה המשותפת',
           'הציונות הדתית',
           'כחול לבן',
           'ישראל ביתנו',
           'הליכוד',
           'מרצ',
           'רעם',
           'יש עתיד',
           'שס',
           'תקוה חדשה']

#
